In [1]:
!pwd
import sys
sys.path.append('/workspace')

/workspace/notebooks


In [2]:
from src.core.db.config import DatabaseEnum
from src.core.db.models.pdf_models import Fincen8300Rev4
from src.core.db.session import DBContext, DbQuery, AppSession

[28/03/2021 01:40:34] [    INFO] [dotenv.main] [_get_stream():77] [PID:51 TID:140278138722112] -> Python-dotenv could not find configuration file /workspace/src/core/env/.local.env.local.


In [3]:
#db = DbQuery(DatabaseEnum.PDF_INGESTION_DB)
#result = db.execute("SELECT * from public.fincen8300_rev4")
from src.core.db.models.pdf_models import Fincen8300Rev4
app_pdf = AppSession(DatabaseEnum.PDF_INGESTION_DB)
session_pdf = app_pdf.instance
result = session_pdf.query(Fincen8300Rev4).all()
print(result)
print(len(result))

[28/03/2021 01:40:34] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:51 TID:140278138722112] -> select version()
[28/03/2021 01:40:34] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:51 TID:140278138722112] -> {}
[28/03/2021 01:40:34] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:51 TID:140278138722112] -> select current_schema()
[28/03/2021 01:40:34] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:51 TID:140278138722112] -> {}
[28/03/2021 01:40:34] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1341] [PID:51 TID:140278138722112] -> SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
[28/03/2021 01:40:34] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1342] [PID:51 TID:140278138722112] -> {}
[28/03/2021 01:40:34] [    INFO] [sqlalchemy.engine.base.Engine] [_cursor_execute():1341] [PID:51 TID:140278138722112] -> SELECT CAST('test unicode returns

[<src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f1f0>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f280>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f550>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f5b0>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f610>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f670>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f6d0>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f730>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f790>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f7f0>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f850>, <src.core.db.models.pdf_models.Fincen8300Rev4 object at 0x7f94ed54f8b0>]
12


In [24]:
from src.core.db.models.pdf_models import IngestionEvent
ingestion_event_id = "17d0e319-c247-432f-9d91-f4259430f25b"
#ingestion_event_id = "99d0e319-c247-432f-9d91-f4259430f25b"  # fake one
result = session_pdf.query(IngestionEvent).filter(IngestionEvent.id == ingestion_event_id).one_or_none()
if result is None:
    print("None found")
else:
    print(result.id)


[28/03/2021 01:56:35] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:51 TID:140278138722112] -> SELECT ingestion_event.id AS ingestion_event_id, ingestion_event.s3_bucket AS ingestion_event_s3_bucket, ingestion_event.s3_key AS ingestion_event_s3_key, ingestion_event.created_at AS ingestion_event_created_at, ingestion_event.updated_at AS ingestion_event_updated_at, ingestion_event.parsing_strategy_type_id AS ingestion_event_parsing_strategy_type_id 
FROM ingestion_event 
WHERE ingestion_event.id = %(id_1)s
[28/03/2021 01:56:35] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:51 TID:140278138722112] -> {'id_1': '17d0e319-c247-432f-9d91-f4259430f25b'}


17d0e319-c247-432f-9d91-f4259430f25b


# Get the fincen document corresponding to the ingestion id

In [32]:
result = session_pdf.query(Fincen8300Rev4).filter(Fincen8300Rev4.ingestion_event_id == ingestion_event_id).one_or_none()
if result is None:
    print("None found")
else:
    print(result.id)
    print(type(result))


[28/03/2021 02:06:58] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:51 TID:140278138722112] -> SELECT fincen8300_rev4.id AS fincen8300_rev4_id, fincen8300_rev4.amends_prior_report AS fincen8300_rev4_amends_prior_report, fincen8300_rev4.suspicious_transaction AS fincen8300_rev4_suspicious_transaction, fincen8300_rev4.multiple_individuals AS fincen8300_rev4_multiple_individuals, fincen8300_rev4.last_name AS fincen8300_rev4_last_name, fincen8300_rev4.first_name AS fincen8300_rev4_first_name, fincen8300_rev4.middle_initial AS fincen8300_rev4_middle_initial, fincen8300_rev4.tin AS fincen8300_rev4_tin, fincen8300_rev4.address AS fincen8300_rev4_address, fincen8300_rev4.city AS fincen8300_rev4_city, fincen8300_rev4.state AS fincen8300_rev4_state, fincen8300_rev4.zip AS fincen8300_rev4_zip, fincen8300_rev4.country AS fincen8300_rev4_country, fincen8300_rev4.occ_prof_bus AS fincen8300_rev4_occ_prof_bus, fincen8300_rev4.identdoc_describe AS fincen8300_rev4_identdoc_de

bed65ea9-5d4e-45f4-9f03-6b924b4b8a54
<class 'src.core.db.models.pdf_models.Fincen8300Rev4'>


In [39]:
from sqlalchemy.inspection import inspect
def convert_obj(obj):
    result = defaultdict(list)
    instance = inspect(obj)
    for key, x in instance.attrs.items():
            result[key].append(x.value)
    return result
    
    
from collections import defaultdict
def query_to_dict(rset):
    result = defaultdict(list)
    for obj in rset:
        instance = inspect(obj)
        for key, x in instance.attrs.items():
            result[key].append(x.value)
    return result
df = pd.DataFrame(convert_obj(result))
df.head()

,ingestion_event,id,amends_prior_report,suspicious_transaction,multiple_individuals,last_name,first_name,middle_initial,tin,address,...,zip_code_brc,nature_of_business_brc,signature_footer,title_footer,contact_name_printed_brc,contact_phone_brc,ingestion_event_id,dob,date_cash_received,date_of_signature_footer
0,<src.core.db.models.pdf_models.IngestionEvent ...,bed65ea9-5d4e-45f4-9f03-6b924b4b8a54,None,None,None,Hammett,Kirk,None,222222223,20 S. Green Hill Ave.,...,None,None,None,None,Ivan Alonzo,None,17d0e319-c247-432f-9d91-f4259430f25b,1962-11-18,1991-11-11,2019-11-22


# Read employee data into data source¶

In [30]:
import pandas as pd
db = DbQuery(DatabaseEnum.MAIN_INGESTION_DB)
result = db.execute("SELECT * from public.employee")
df_employee = pd.DataFrame(result.fetchall())
df_employee.columns = result.keys()
df_employee.head(20)

[28/03/2021 02:02:34] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:51 TID:140278138722112] -> SELECT * from public.employee
[28/03/2021 02:02:34] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:51 TID:140278138722112] -> {}


,id,prefix_name,first_name,last_name,middle_name,suffix_name,date_of_birth,dod_id,ssn
0,dcd9407d-47a8-420f-a214-6645c52f002a,None,Diane,Meier,P,Sr.,2057-02-28,7186,686280445
1,9d2b8117-0ce9-4637-ae1f-746c7d72530e,Ms.,Samantha,Young,,Sr.,1980-07-13,8955,195501688
2,7f179f45-97cc-45f8-b8f5-0bf1c7c38e1c,None,Adrian,Frandle,,None,1982-12-20,5805,715397798
3,20d2496d-81b0-4915-ad15-ca7111e3d1a3,Ms.,Christine,Johnson,Marie,None,1991-04-07,6424,105019573
4,87f2d705-80c3-4e7a-9076-90e5204dfa70,None,Kaisheng,Yao,,Jr.,1972-02-17,8930,503834214
5,7402b0f5-4ec8-4d64-ae9a-3912a445ef3f,Mr.,Mayandran,Mathen,,None,1975-09-05,7089,197982969
6,87331e3e-eec4-466b-862a-6844954db3b5,None,Max,Roesler,J,None,1988-12-18,7931,104024454
7,9834c863-5f50-486b-8bd3-4a6328cd7627,None,Yasmin,Alpay,,None,2068-09-05,4615,455023577
8,5feac035-46bb-41ab-8e94-806bc57df2c1,None,Jieming,Gao,,Sr.,1977-11-13,5882,189200224
9,69eaf532-8069-4e92-8942-5fa43f14df21,None,Jin,Jang,,None,2069-01-03,6394,488780128


In [45]:
# only reasonable way to get into a dataframe
query = session_pdf.query(Fincen8300Rev4).filter(Fincen8300Rev4.ingestion_event_id == ingestion_event_id).statement
df = pd.read_sql(query, app_pdf._engine)
df.head()

[28/03/2021 02:33:18] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:51 TID:140278138722112] -> select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
[28/03/2021 02:33:18] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:51 TID:140278138722112] -> {'name': 'SELECT fincen8300_rev4.id, fincen8300_rev4.amends_prior_report, fincen8300_rev4.suspicious_transaction, fincen8300_rev4.multiple_individuals, fincen8 ... (2168 characters truncated) ... ate_cash_received, fincen8300_rev4.date_of_signature_footer \nFROM fincen8300_rev4 \nWHERE fincen8300_rev4.ingestion_event_id = :ingestion_event_id_1'}
[28/03/2021 02:33:18] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:51 TID:140278138722112] -> SELECT fincen8300_rev4.id, fincen8300_rev4.amends_prior_report, fincen8300_rev4.suspicious_transaction, fincen8300_rev4.multiple_individuals, f

,id,amends_prior_report,suspicious_transaction,multiple_individuals,last_name,first_name,middle_initial,tin,address,city,...,zip_code_brc,nature_of_business_brc,signature_footer,title_footer,contact_name_printed_brc,contact_phone_brc,ingestion_event_id,dob,date_cash_received,date_of_signature_footer
0,bed65ea9-5d4e-45f4-9f03-6b924b4b8a54,None,None,None,Hammett,Kirk,None,222222223,20 S. Green Hill Ave.,Ephrata,...,None,None,None,None,Ivan Alonzo,None,17d0e319-c247-432f-9d91-f4259430f25b,1962-11-18,1991-11-11,2019-11-22


# Map The Columns

In [31]:
from src.sources.structured_data_source import StructuredDataSource
from src.mapping.pdfs.pdf_field_name_classifier import FieldNameClassifier
from src.mapping.pdfs.pdf_field_label_catalog import FieldLabelCatalog
from src.mapping.pdfs.pseudofield_generator import PseudofieldGenerator

pseudofield_generator = PseudofieldGenerator(fincen)
pseudofield_generator.generate()

def _create_column_relations_for(source, target):
    """Create column relations from canonical column identifiers."""
    gold_id_info = FieldNameClassifier.get_id_info_from_df(target.get_data())
    data_id_info = FieldNameClassifier.get_id_info_from_df(source.get_data())
    for identifier in FieldLabelCatalog:
        if identifier in gold_id_info and identifier in data_id_info:
            g_id = gold_id_info[identifier]
            d_id = data_id_info[identifier]
            source.create_column_relation(
                d_id.field_name, g_id.field_name, target
            )
            print(
                "New relation detected: %s" % str(source.column_relations[-1])
            )

NameError: name 'fincen' is not defined

In [49]:
db = DbQuery(DatabaseEnum.PDF_INGESTION_DB)
result = db.execute("SELECT * from public.fincen8300_rev4 limit 10")
df = pd.DataFrame(result.fetchall())
df.columns = result.keys()
print(df.shape)

[28/03/2021 02:49:46] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1235] [PID:51 TID:140278138722112] -> SELECT * from public.fincen8300_rev4 limit 10
[28/03/2021 02:49:46] [    INFO] [sqlalchemy.engine.base.Engine] [_execute_context():1237] [PID:51 TID:140278138722112] -> {}


(10, 69)
